In [13]:
# author:syt & monkeydc
# last edit time:

import pandas as pd
#import sys
import os
import numpy as np
import openpyxl
import math


os.chdir("C:/Users/哈哈/PycharmProjects/datasets")#地址修改成你自己的c盘那个，我暂时没改好这个，在看下代码
#print(os.getcwd())
#sys.path.append()

#生成格式化列表，可以用于共现计算
def generatelist(path):
    a = []
    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                c.append(a[i][j])
        b.append(c)
    return b

#建立字典
def generateDict(path,b1):
    a = []

    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        # c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                # c.append(a[i][j])
                b.append(a[i][j])
    data_dict = pd.DataFrame(b)
    data_dict.columns = ['label']
    label_list = list(set(data_dict['label'].values.tolist()))
    labeldic = {}
    countdic = {}
    pos = 0
    for i in label_list:
        pos = pos + 1
        labeldic[i] = int(pos)
    #return labeldic
    appeardict = {}  # 各个标签为键，出现在哪些文本（将文本按顺序排列，出现的序号添加到列表中）为值，构造字典
    for l in label_list:
        appeardict[l]=[]
    i = 0
    for w in b1:
        for w1 in w:
            appeardict[w1].append(i)
        i += 1
    for m in label_list:
        countdic[m] = len(appeardict[m])
    print(len(countdic))
    return countdic#计算每个标签出现的次数用字典保存


b = generatelist("./aapd/tag")
countdic = generateDict("./aapd/tag",b)
print(countdic)

def generateFMetrix(len):
    metrix = np.eye(len)
    return metrix







def countPMI(path):
    # 计算共现矩阵
    dict = generateDict(path,b)
    list = generatelist(path)
    length = len(dict)
    #metrix = generateFMetrix(length + 1)
    metrix = []#矩阵总是报错，因为盲猜矩阵固定了单元格内的数据格式，为能够保存中间结果使用list
    for i in range(length+1):
        metrix.append([])
        for j in range(length+1):
            metrix[i].append(0)

    for i in range(54):  # 字典里每一个词覆盖
        i += 1
        for j in range(54):
            j += 1
            z = 0
            for l in range(len(list)):  # 便利全部list
                for m in range(len(list[l])):  # 遍历list当中每一个可能，判断i是否在其中
                    if dict[i] == list[l][m]:
                        for n in range(len(list[l])):  # 遍历list当中每一个可能,判断j是否在其中，是则z加1
                            if dict[j] == list[l][n]:
                                z += 1
                            else:
                                pass
                    else:
                        pass
                # print(z)
            metrix[i][j] = z
    for i in range(length+1):
        if i ==0:
            metrix[i][i] = 0
        else:
            metrix[i][0] = dict[i]#给每一行每一列搭上标记
            metrix[0][i] = dict[i]#矩阵内部没办法打上string？总有报错说string没法convert in to float，所以不应该用这个方法存储
                                  #其实list可以考虑下，然后按照索引输出
    #没有计算完pmi，因为共现次数统计之后还需要
    return metrix




def writeToExcel(file_path, new_list):
    # total_list = [['A', 'B', 'C', 'D', 'E'], [1, 2, 4, 6, 8], [4, 6, 7, 9, 0], [2, 6, 4, 5, 8]]
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = '明细'
    for r in range(len(new_list)):
        for c in range(len(new_list[0])):
            ws.cell(r + 1, c + 1).value = new_list[r][c]
            # excel中的行和列是从1开始计数的，所以需要+1
    wb.save(file_path)  # 注意，写入后一定要保存
    print("成功写入文件: " + file_path + " !")
    return None


def PMI(metrix):
    for x,y in zip(label_list,label_list):
        i=0
        j=0
        for i,j in zip(range(55),range(55)):
            if x == metrix[i][0] and y == metrix[0][j]:
                con=metrix[i][j]
                i += 1
                j += 1
        conx=countdic[x]
        cony=countdic[y]
        a = con / (conx * cony)
        pmi = np.log(a)
        metrixP=generateFMetrix(55)
        metrixP[0][1:]=label_list
        metrixP[1:][0]=label_list
        for p, q in range(55):
            if p == metrixP[i][0] and q == metrixP[0][j]:
                metrixP[p][q]=pmi
                p += 1
                q += 1
    return metrixP

def write(file_path, new_list):
    # total_list = [['A', 'B', 'C', 'D', 'E'], [1, 2, 4, 6, 8], [4, 6, 7, 9, 0], [2, 6, 4, 5, 8]]
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = '明细'
    for r in range(len(new_list)):
        for c in range(len(new_list[0])):
            ws.cell(r + 1, c + 1).value = new_list[r][c]
            # excel中的行和列是从1开始计数的，所以需要+1
    wb.save(file_path)  # 注意，写入后一定要保存
    print("成功写入文件: " + file_path + " !")
    return None

if __name__ == '__main__':
    metrix = countPMI('./aapd/tag')
    file_path = './aapd/metrixP.csv'
    b=generatelist('./aapd/tag')
    label_list=generateDict('./aapd/tag',b)
    countdic=generateDict('./aapd/tag',b)
    metrixP=PMI(metrix)
    #metrixP=PMI(metrix,countdic,label_list)
    write(file_path, metrixP)

54
{'q-bio.nc': 416, 'cs.ir': 2058, 'cs.lg': 8153, 'cs.lo': 3581, 'cs.cl': 2882, 'cs.ne': 2055, 'cs.si': 4074, 'cs.dl': 880, 'math.na': 965, 'cs.gt': 1749, 'math.nt': 390, 'cs.dm': 4142, 'nlin.ao': 358, 'cs.cv': 2817, 'cs.pf': 702, 'cs.it': 17489, 'cs.ni': 3135, 'math.pr': 1214, 'cs.db': 1291, 'cs.ds': 4751, 'cs.ms': 491, 'cond-mat.stat-mech': 678, 'cs.se': 1281, 'quant-ph': 1617, 'math.co': 3415, 'cs.ai': 5102, 'cs.pl': 1409, 'stat.me': 506, 'math.it': 17489, 'math.lo': 676, 'cs.ce': 947, 'cs.dc': 2299, 'cs.sc': 402, 'stat.th': 895, 'cs.cr': 2772, 'cs.hc': 1021, 'physics.soc-ph': 3390, 'cs.cc': 3123, 'stat.ap': 642, 'math.oc': 3096, 'cond-mat.dis-nn': 449, 'cmp-lg': 871, 'cs.cy': 1611, 'cs.ma': 970, 'cs.na': 1072, 'q-bio.qm': 382, 'cs.ro': 937, 'cs.cg': 1102, 'math.st': 895, 'cs.fl': 933, 'stat.ml': 5033, 'cs.sy': 2286, 'cs.mm': 726, 'physics.data-an': 394}
54


KeyError: 1

In [ ]:
if __name__ == '__main__':
    metrix = countPMI('./aapd/tag')
    file_path = './results/metrix.csv'
    b=generatelist('./aapd/tag')
    label_list=generateDict('./aapd/tag',b)
    countdic=generateDict('./aapd/tag',b)
    print(len(countdic))
    #metrixP=PMI(metrix,countdic,label_list)
    #writeToExcel(file_path, metrix)

In [38]:


import pandas as pd
#import sys
import os
import numpy as np
import openpyxl


os.chdir("C:/Users/哈哈/PycharmProjects/datasets")#地址修改成你自己的c盘那个，我暂时没改好这个，在看下代码
#print(os.getcwd())
#sys.path.append()

#生成格式化列表，可以用于共现计算
def generatelist(path):
    a = []
    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                c.append(a[i][j])
        b.append(c)
    return b

#建立字典
def generateDict(path):
    a = []
    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        # c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                # c.append(a[i][j])
                b.append(a[i][j])
    data_dict = pd.DataFrame(b)
    data_dict.columns = ['label']
    label_list = list(set(data_dict['label'].values.tolist()))
    labeldic = {}
    pos = 0
    for i in label_list:
        pos = pos + 1
        labeldic[pos] = str(i)
    return labeldic

def generateFMetrix(len):
    metrix = np.eye(len)
    return metrix

def countPMI(path):

    # 计算共现矩阵
    dict = generateDict(path)
    list = generatelist(path)
    length = len(dict)
    #metrix = generateFMetrix(length + 1)
    metrix = []#矩阵总是报错，因为盲猜矩阵固定了单元格内的数据格式，为能够保存中间结果使用list
    for i in range(length+1):
        metrix.append([])

        for j in range(length+1):
            
            metrix[i].append(0)

    for i in range(54):  # 字典里每一个词覆盖
        i += 1
        for j in range(54):
            j += 1
            z = 0
            for l in range(len(list)):  # 便利全部list
                for m in range(len(list[l])):  # 遍历list当中每一个可能，判断i是否在其中
                    if dict[i] == list[l][m]:
                        for n in range(len(list[l])):  # 遍历list当中每一个可能,判断j是否在其中，是则z加1
                            if dict[j] == list[l][n]:
                                z += 1
                            else:
                                pass
                    else:
                        pass
                #print(z)
            metrix[i][j] = z
    
    for i in range(length+1):
        if i ==0:
            metrix[i][i] = 0
        else:
            metrix[i][0] = dict[i]#给每一行每一列搭上标记
            metrix[0][i] = dict[i]#矩阵内部没办法打上string？总有报错说string没法convert in to float，所以不应该用这个方法存储
    print(metrix) 
    #其实list可以考虑下，然后按照索引输出
    #没有计算完pmi，因为共现次数统计之后还需要
    return metrix

def writeToExcel(file_path, new_list):
    # total_list = [['A', 'B', 'C', 'D', 'E'], [1, 2, 4, 6, 8], [4, 6, 7, 9, 0], [2, 6, 4, 5, 8]]
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = '明细'
    for r in range(len(new_list)):
        for c in range(len(new_list[0])):
            ws.cell(r + 1, c + 1).value = new_list[r][c]
            # excel中的行和列是从1开始计数的，所以需要+1
    wb.save(file_path)  # 注意，写入后一定要保存
    print("成功写入文件: " + file_path + " !")
    return None


metrix=countPMI("./aapd/tag")
print(metrix)


[[0, 'q-bio.nc', 'cs.ir', 'cs.lg', 'cs.lo', 'cs.cl', 'cs.ne', 'cs.si', 'cs.dl', 'math.na', 'cs.gt', 'math.nt', 'cs.dm', 'nlin.ao', 'cs.cv', 'cs.pf', 'cs.it', 'cs.ni', 'math.pr', 'cs.db', 'cs.ds', 'cs.ms', 'cond-mat.stat-mech', 'cs.se', 'quant-ph', 'math.co', 'cs.ai', 'cs.pl', 'stat.me', 'math.it', 'math.lo', 'cs.ce', 'cs.dc', 'cs.sc', 'stat.th', 'cs.cr', 'cs.hc', 'physics.soc-ph', 'cs.cc', 'stat.ap', 'math.oc', 'cond-mat.dis-nn', 'cmp-lg', 'cs.cy', 'cs.ma', 'cs.na', 'q-bio.qm', 'cs.ro', 'cs.cg', 'math.st', 'cs.fl', 'stat.ml', 'cs.sy', 'cs.mm', 'physics.data-an'], ['q-bio.nc', 416, 7, 68, 2, 21, 125, 15, 0, 2, 1, 0, 1, 12, 72, 0, 31, 0, 2, 7, 6, 1, 9, 1, 3, 1, 80, 0, 7, 31, 1, 16, 7, 0, 2, 0, 48, 19, 2, 10, 4, 25, 0, 4, 4, 2, 23, 13, 2, 2, 0, 46, 5, 1, 9], ['cs.ir', 7, 2058, 449, 1, 532, 36, 266, 256, 4, 9, 0, 12, 4, 153, 4, 36, 18, 3, 175, 100, 1, 11, 27, 8, 0, 279, 6, 8, 36, 0, 8, 33, 0, 6, 42, 120, 137, 9, 19, 12, 6, 0, 78, 7, 9, 14, 2, 9, 6, 1, 184, 2, 81, 24], ['cs.lg', 68, 449, 81

In [83]:
import pandas as pd

data=pd.read_excel(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\metrix.csv",index_col=0)
#data.head()
def PMI(data):
    i=0
    j=0
    list1=data.columns.tolist()
    list2=data.index.tolist()
    for i in range(54):
        for j in range(54):
            conx=countdic[list1[i]]
            cony=countdic[list2[j]]
    #for i,j in zip(range(54),range(54)):
            #if list1[i] == metrix[i][0] and list2[j] == metrix[0][j]:
            con=data.iloc[i,j]
             
        #conx=countdic[x]
        
            a = con / (conx * cony)
            pmi = np.log(a)
            data.iloc[i,j]=pmi
           
    #metrixP=generateFMetrix(54)
        #metrixP[0][1:]=label_list
        #metrixP[1:][0]=label_list
    print(data)
    pd.to_csv(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\pmicsv.csv")
    #return metrixP

def generatelist(path):
    a = []
    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                c.append(a[i][j])
        b.append(c)
    return b

#建立字典
def generateDict(path,b1):
    a = []

    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        # c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                # c.append(a[i][j])
                b.append(a[i][j])
    data_dict = pd.DataFrame(b)
    data_dict.columns = ['label']
    label_list = list(set(data_dict['label'].values.tolist()))
    labeldic = {}
    countdic = {}
    pos = 0
    for i in label_list:
        pos = pos + 1
        labeldic[i] = int(pos)
    #return labeldic
    appeardict = {}  # 各个标签为键，出现在哪些文本（将文本按顺序排列，出现的序号添加到列表中）为值，构造字典
    for l in label_list:
        appeardict[l]=[]
    i = 0
    for w in b1:
        for w1 in w:
            appeardict[w1].append(i)
        i += 1
    for m in label_list:
        countdic[m] = len(appeardict[m])
    print(len(countdic))
    return countdic,label_list#计算每个标签出现的次数用字典保存


b = generatelist("./aapd/tag")
countdic,label_list = generateDict("./aapd/tag",b)
print(countdic)

def generateFMetrix(len):
    metrix = np.eye(len)
    return metrix
metrixP=PMI(data)
#metrixP.to_csv("./pmicsv.csv")
#print(metrixP)

54
{'q-bio.nc': 416, 'cs.ir': 2058, 'cs.lg': 8153, 'cs.lo': 3581, 'cs.cl': 2882, 'cs.ne': 2055, 'cs.si': 4074, 'cs.dl': 880, 'math.na': 965, 'cs.gt': 1749, 'math.nt': 390, 'cs.dm': 4142, 'nlin.ao': 358, 'cs.cv': 2817, 'cs.pf': 702, 'cs.it': 17489, 'cs.ni': 3135, 'math.pr': 1214, 'cs.db': 1291, 'cs.ds': 4751, 'cs.ms': 491, 'cond-mat.stat-mech': 678, 'cs.se': 1281, 'quant-ph': 1617, 'math.co': 3415, 'cs.ai': 5102, 'cs.pl': 1409, 'stat.me': 506, 'math.it': 17489, 'math.lo': 676, 'cs.ce': 947, 'cs.dc': 2299, 'cs.sc': 402, 'stat.th': 895, 'cs.cr': 2772, 'cs.hc': 1021, 'physics.soc-ph': 3390, 'cs.cc': 3123, 'stat.ap': 642, 'math.oc': 3096, 'cond-mat.dis-nn': 449, 'cmp-lg': 871, 'cs.cy': 1611, 'cs.ma': 970, 'cs.na': 1072, 'q-bio.qm': 382, 'cs.ro': 937, 'cs.cg': 1102, 'math.st': 895, 'cs.fl': 933, 'stat.ml': 5033, 'cs.sy': 2286, 'cs.mm': 726, 'physics.data-an': 394}


<ipython-input-83-03aee3279aba>:21: RuntimeWarning: divide by zero encountered in log
  pmi = np.log(a)


                      stat.ml    stat.ap  cond-mat.dis-nn      cs.ce  \
0                                                                      
stat.ml             -8.523772 -10.105558       -11.411919 -12.009775   
stat.ap            -10.105558  -6.464588             -inf -10.226845   
cond-mat.dis-nn    -11.411919       -inf        -6.107023       -inf   
cs.ce              -12.009775 -10.226845             -inf  -6.853299   
q-bio.nc           -10.725815 -10.192688        -8.918832 -10.111396   
cs.fl                    -inf       -inf             -inf -12.082266   
cs.pf              -13.691411 -11.226762       -12.660956 -10.229179   
cs.ma              -12.510696 -11.732446       -10.904877 -10.204235   
cs.mm              -12.713426 -11.665844             -inf -13.440849   
math.it            -12.151061 -11.278089       -11.469631 -13.255331   
stat.th             -9.718476 -10.428199       -12.210699 -12.551511   
cs.cr              -12.713426 -11.396180       -12.935735 -12.58

AttributeError: module 'pandas' has no attribute 'to_csv'

In [85]:
os.path.abspath('.')

'C:\\Users\\哈哈\\PycharmProjects\\datasets'

In [87]:
import pandas as pd
import numpy as np
data = pd.read_excel(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\metrix.csv",index_col=0)#共现矩阵文件的路径
list=data.values.tolist()
#print(list[0][0])
#print(data)
#print(list[1][1])

def PMI(data):
    i = 0
    j = 0
    list1 = data.columns.tolist()
    list2 = data.index.tolist()
    for i in range(54):
        for j in range(54):
            conx = countdic[list1[i]]
            cony = countdic[list2[j]]
            con = data.iloc[i, j]

            # conx=countdic[x]

            a = con / (conx * cony)
            pmi = np.log(a)
            data.iloc[i, j] = pmi

    # metrixP=generateFMetrix(54)
    # metrixP[0][1:]=label_list
    # metrixP[1:][0]=label_list
    #pd.to_csv(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\pmicsv.csv")#保存计算好的PMI
    #print(data)
    # return metrixP

def newPMI(data,list):
    i = 0
    l = len(data)
    for i in range(len(list)):
        conx = list[i][i]
        px = conx/l
        i += 1
        j = 0
        for j in range(len(list)):
            cony = list[j][j]
            py = cony/l
            con = list[i][j]
            pxy = con/l
            j += 1

            #print(con)


            if con == 0:
                pmi=0
                print("b")
            else:
                #pmi = np.log(pxy / (px * py))
                #list[i][j] = pmi
    #print("a")
    #pd.to_csv(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\newpmicsv.csv")  # 保存计算好的PMI

def generatelist(path):
    a = []
    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                c.append(a[i][j])
        b.append(c)
    return b


# 建立字典
def generateDict(path, b1):
    a = []

    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        # c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                # c.append(a[i][j])
                b.append(a[i][j])
    data_dict = pd.DataFrame(b)
    data_dict.columns = ['label']
    label_list = list(set(data_dict['label'].values.tolist()))
    labeldic = {}
    countdic = {}
    pos = 0
    for i in label_list:
        pos = pos + 1
        labeldic[i] = int(pos)
    # return labeldic
    appeardict = {}  # 各个标签为键，出现在哪些文本（将文本按顺序排列，出现的序号添加到列表中）为值，构造字典
    for l in label_list:
        appeardict[l] = []
    i = 0
    for w in b1:
        for w1 in w:
            appeardict[w1].append(i)
        i += 1
    for m in label_list:
        countdic[m] = len(appeardict[m])
    print(len(countdic))
    return countdic, label_list  # 计算每个标签出现的次数用字典保存


#b = generatelist("./aapd/tag")
#countdic, label_list = generateDict("./aapd/tag", b)
#print(countdic)


c = newPMI(data,list)




132
642
0
22
10
0
6
5
4
142
17
20
41
62
137
5
18
8
54
18
5
13
110
30
14
0
2
0
26
35
8
39
106
0
1
0
6
0
9
19
3
9
0
0
3
2
0
2
7
10
17
21
31
142
25
0
449
0
25
0
1
8
0
82
2
3
2
7
42
3
16
176
23
47
2
2
108
3
5
16
0
0
3
4
0
3
138
0
72
1
2
0
32
6
9
37
1
1
31
0
0
28
2
2
2
27
0
82
29
22
0
947
16
5
24
34
1
29
3
9
6
35
128
14
6
6
84
7
26
7
14
16
125
5
20
1
7
38
4
7
16
0
13
24
27
1
65
8
6
95
42
3
9
70
59
23
82
92
3
2
39
29
46
10
25
16
416
0
0
4
1
31
2
0
0
72
68
1
2
9
80
0
1
48
15
4
23
3
0
1
21
4
13
7
19
0
2
2
7
0
6
7
1
125
2
0
12
2
1
1
7
2
2
9
5
31
0
0
0
5
0
933
1
5
0
12
0
7
0
0
8
47
6
4
16
5
28
0
0
2
5
37
38
21
16
14
4
0
0
0
145
7
14
12
63
1
138
3
397
4
1
0
1
151
22
1
0
0
27
12
4
6
1
24
0
1
702
4
14
98
4
19
3
5
10
6
72
2
9
257
30
1
2
1
3
0
42
0
0
23
1
1
0
0
9
1
35
0
30
4
1
6
22
1
1
6
28
11
183
18
4
1
27
98
18
5
8
34
4
5
4
970
4
35
4
24
6
5
59
228
12
13
270
43
32
19
69
35
3
2
7
7
11
104
80
1
69
0
57
9
3
0
34
7
13
36
71
1
38
3
3
16
59
3
4
1
136
35
11
4
0
1
1
0
14
4
726
80
1
140
13
208
38
2
1
0
29
1

IndexError: list index out of range

In [91]:
a=0.123456
a1=round(a,3)
print(a1)

0.123


In [ ]:
import pandas as pd
import numpy as np
da = pd.read_excel(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\metrix.csv",index_col=0)#共现矩阵文件的路径
tag = pd.read_csv(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\datacsv.csv")
list=da.values.tolist()
#print(list[0][0])
#print(len(tag))
#print(list[1][1])

def PMI(data):
    i = 0
    j = 0
    list1 = data.columns.tolist()
    list2 = data.index.tolist()
    for i in range(54):
        for j in range(54):
            conx = countdic[list1[i]]
            cony = countdic[list2[j]]
            con = data.iloc[i, j]

            # conx=countdic[x]

            a = con / (conx * cony)
            pmi = np.log(a)
            data.iloc[i, j] = pmi

    # metrixP=generateFMetrix(54)
    # metrixP[0][1:]=label_list
    # metrixP[1:][0]=label_list
    #pd.to_csv(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\pmicsv.csv")#保存计算好的PMI
    #print(data)
    # return metrixP

def newPMI(tag,list):
    l = len(tag)
    for i in range(len(list)):
        conx = list[i][i]
        px = conx/l
        for j in range(len(list)):
            cony = list[j][j]
            py = cony/l
            con = list[i][j]
            pxy = con/l
            #print(con)
            if con != 0:
                pmi = (pxy / (px * py))
                list[i][j] = pmi
            else:
                list[i][j] = 0
    test = pd.DataFrame(data=list)
    test.to_csv(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\newpmicsv1.csv")  # 保存计算好的PMI

def generatelist(path):
    a = []
    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                c.append(a[i][j])
        b.append(c)
    return b


# 建立字典
def generateDict(path, b1):
    a = []

    with open(path, 'r') as f:
        for line in f:
            line = line.replace('[', '')
            line = line.replace(']', '')
            a.append(line.strip('\n').split("'"))
    b = []
    for i in range(len(a)):
        # c = []
        for j in range(len(a[i])):
            if j % 2 == 1:
                # c.append(a[i][j])
                b.append(a[i][j])
    data_dict = pd.DataFrame(b)
    data_dict.columns = ['label']
    label_list = list(set(data_dict['label'].values.tolist()))
    labeldic = {}
    countdic = {}
    pos = 0
    for i in label_list:
        pos = pos + 1
        labeldic[i] = int(pos)
    # return labeldic
    appeardict = {}  # 各个标签为键，出现在哪些文本（将文本按顺序排列，出现的序号添加到列表中）为值，构造字典
    for l in label_list:
        appeardict[l] = []
    i = 0
    for w in b1:
        for w1 in w:
            appeardict[w1].append(i)
        i += 1
    for m in label_list:
        countdic[m] = len(appeardict[m])
    print(len(countdic))
    return countdic, label_list  # 计算每个标签出现的次数用字典保存


#b = generatelist("./aapd/tag")
#countdic, label_list = generateDict("./aapd/tag", b)
#print(countdic)


c = newPMI(tag,list)




In [ ]:
import pandas as pd
import numpy as np
da = pd.read_excel(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\metrix.csv",index_col=0)#共现矩阵文件的路径
tag = pd.read_csv(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\datacsv.csv")#tag的csv文件路径
list = da.values.tolist()

def newPMI(tag,list):
    l = len(tag)
    for i in range(len(list)):
        conx = list[i][i]
        px = conx/l
        for j in range(len(list)):
            cony = list[j][j]
            py = cony/l
            con = list[i][j]
            pxy = con/l
            #print(con)
            if con != 0:
                pmi = (pxy / (px * py))
                list[i][j] = pmi
            else:
                list[i][j] = 0
    test = pd.DataFrame(data=list)
    test.to_csv(r"C:\Users\哈哈\PycharmProjects\datasets\aapd\newpmicsv1.csv")  # 保存计算好的PMI


c = newPMI(tag,list)


